In [ ]:
import pandas as pd

## Cleaning and preparing the Bechdel score file
Original file: bechdel_movies_2023_FEB.csv

In [ ]:
bechdel = pd.read_csv('bechdel_movies_2023_FEB.csv')

In [ ]:
# Checking for duplicates (no duplicates)
bechdel.duplicated().sum()

In [ ]:
# Stripping leading and trailing whitespace in all string variables
bechdel = bechdel.apply(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# Removing rows where dubious = 1 (these are contested scores, about 10% of the dataset)
bechdel = bechdel[bechdel['dubious'] == 0]

In [ ]:
# Moving the year into the title name (as in IMDB) to reduce duplicate names
title_list = list(bechdel['title'])
year_list = list(bechdel['year'])

title_and_year_list = [f'{title_list[i]} ({year_list[i]})' for i in range(len(title_list))]
bechdel['title'] = title_and_year_list

# Check: Number has drastically reduced (but there are still 3 values -> individual treatment below)
bechdel['title'].value_counts().head(7)

In [ ]:
# Individual treatment of title 1 (we drop both duplicates as the ratings are different, i.e., "dubious")
bechdel = bechdel.drop([8892, 9111])

In [ ]:
# Individual check for title 2 (we drop one duplicate but keep the entry as the rating is the same)
bechdel = bechdel.drop(951)

In [ ]:
# Individual treatment for title 3 (we drop both duplicates as the ratings are different, i.e., "dubious")
bechdel = bechdel.drop([259, 270])

In [ ]:
# Dropping columns...
# - "year" to avoid redundancy (year information will be provided in the movies_info table)
# - "imdbid" as sadly the IMDB ID is not provided in the IMDB movie list we use for the movies_info table
# - "submitterid" as this is information we do not need for our business case
# - "visible" as every value here is == 1
# - "dubious" as we removed all dubious == 1

bechdel = bechdel.drop(['year', 'imdbid', 'submitterid', 'visible', 'dubious'],axis=1)

In [ ]:
# Creating a pass/not pass column for the bechdel test result
bechdel['bechdel_pass'] = bechdel['rating'].apply(lambda x: 1 if x == 3 else 0)

In [ ]:
# Extracting only the year from the date
bechdel['date'] = bechdel['date'].apply(lambda x: int(x[0:4]))

In [ ]:
# Re-ordering and renaming
bechdel = bechdel[['id','title','rating','bechdel_pass', 'date']]
bechdel.columns = ['bechdel_ID', 'title', 'bechdel_score', 'bechdel_pass', 'rating_year']

In [ ]:
# Ordering the file after ID and re-indexing
bechdel = bechdel.sort_values(by=['bechdel_ID']).reset_index(drop = True)

In [ ]:
# Checking if data types are correct (all correct)
bechdel.dtypes

In [ ]:
# Checking for missing values (none)
bechdel.isna().sum()

In [ ]:
# Adding the movie_info_ID as foreign key - calling up the cleaned movie_info table
imdb = pd.read_csv('imdb_clean.csv')

In [ ]:
# Merging (outer join)
bechdel_w_foreign_key = pd.merge(bechdel, imdb, on='title', how='left')

In [ ]:
# Dropping unnecessary columns
bechdel_w_foreign_key = bechdel_w_foreign_key.drop(['year', 'imdb_score', 'original_language', 'budget', 'revenue', 'country'],axis=1)

In [ ]:
# Saving the cleaned bechdel file with movie_info_ID included
bechdel_w_foreign_key.to_csv('bechdel.csv', index=False)

## Cleaning and preparing the Bechdel score file
Original file: bechdel_movies_2023_FEB.csv

In [ ]:
imdb = pd.read_csv('imdb_movies.csv', skipinitialspace = True)

In [ ]:
# Dropping unnecessary columns
imdb.drop(["genre", "overview", "crew", "orig_title","status",], axis=1, inplace=True)

In [ ]:
# Rename columns
imdb.columns = ['title', 'year', 'imdb_score', 'original_language', 'budget', 'revenue','country']

In [ ]:
# Transforming datatypes
imdb["budget"] = imdb["budget"].astype(int)
imdb["revenue"] = imdb["revenue"].astype(int)
imdb["year"] = imdb["year"].astype(str)

In [ ]:
# Creating an ID (as primary key)
imdb['movie_info_ID'] = range(1, len(imdb) + 1)

In [ ]:
# Extracting the year out of the former date / now year column
imdb['year_only'] = imdb['year'].apply(lambda x: x.split('/')[2] if type(x)==str else x)

In [ ]:
# Re-ordering columns
new_column_order = ['movie_info_ID','year_only', 'title', 'imdb_score', 'original_language', 'budget', 'revenue', 'country']

imdb = imdb[new_column_order]

In [ ]:
# Renaming (new) year column
imdb = imdb.rename(columns={"year_only":"year"})

In [ ]:
# Moving the year into the title name (as in IMDB)
title_list = list(imdb['title'])
year_list = list(imdb['year'])
title_and_year_list = [f'{title_list[i]} ({year_list[i]})' for i in range(len(title_list))]
imdb['title'] = title_and_year_list

In [ ]:
# Dropping duplicates (after some previous exploration of their nature - first duplicate seemed to be most reliable)
imdb.drop_duplicates(subset=['title'], inplace= True)

In [ ]:
# Cleaning up the title column (some whitespaces were left)
imdb['title'] = imdb['title'].apply(lambda x: x.replace(" )", ")"))

In [ ]:
# Transforming inappropriate datatypes
imdb["year"] = imdb["year"].astype(int)
imdb["imdb_score"] = imdb["imdb_score"].astype(int)

In [ ]:
# Check: Datatypes are now correct
imdb.dtypes

In [ ]:
# Checking for missing values (none)
imdb.isna().sum()

In [ ]:
# Re-ordering columns
new_column_order_2 = ['movie_info_ID','title', 'year', 'imdb_score', 'original_language', 'budget', 'revenue', 'country']
imdb = imdb[new_column_order_2]

In [ ]:
imdb.to_csv('movie_info.csv', index=False)

## Preparing the additional two tables related to genre information
Note: The first steps were the same as above, but with the genres column left in place. Changes are marked with comments.

In [ ]:
df = pd.read_csv('imdb_movies.csv', skipinitialspace = True)

In [ ]:
df.drop(["overview", "crew", "orig_title","status",], axis=1, inplace=True)

In [ ]:
# Here, genre is left in the columns (we excluded it at this point when preparing the movie_info file above)
df.columns = ['title', 'year', 'imdb_score', 'genre','original_language', 'budget', 'revenue','country']

In [ ]:
df["budget"] = df["budget"].astype(int)
df["revenue"] = df["revenue"].astype(int)
df["year"] = df["year"].astype(str)

In [ ]:
df['movie_info_ID'] = range(1, len(df) + 1)

In [ ]:
df['year_only'] = df['year'].apply(lambda x: x.split('/')[2] if type(x)==str else x)

In [ ]:
new_column_order = ['movie_info_ID','year_only', 'title', 'imdb_score', 'original_language', 'budget', 'revenue', 'country', 'genre']
df = df[new_column_order]

In [ ]:
df = df.rename(columns={"year_only":"year"})

In [ ]:
title_list = list(df['title'])
year_list = list(df['year'])
title_and_year_list = [f'{title_list[i]} ({year_list[i]})' for i in range(len(title_list))]
df['title'] = title_and_year_list

In [ ]:
df.drop_duplicates(subset=['title'], inplace= True)

In [ ]:
df['title'] = df['title'].apply(lambda x: x.replace(" )", ")"))

In [ ]:
df["year"] = df["year"].astype(int)
df["imdb_score"] = df["imdb_score"].astype(int)

In [ ]:
# Replacing missing values in genre column (imputing)
df['genre'] = df['genre'].fillna('Other/unspecified')

In [ ]:
new_column_order_2 = ['movie_info_ID','title', 'year', 'imdb_score', 'original_language', 'budget', 'revenue', 'country', 'genre']
df = df[new_column_order_2]

In [ ]:
df_clean = df.drop(["genre"], axis=1)

#### Additional transformations/preparations

In [ ]:
# First reduction of columns
movie_genre = df.copy()
movie_genre = movie_genre[['movie_info_ID', 'title', 'genre']]

In [ ]:
# Getting rid of whitespace-type spaces in the genre column (otherwise leads to issues later)
movie_genre['genre'] = movie_genre['genre'].apply(lambda x: x.replace("\xa0","") if type(x)==str else x)

In [ ]:
# Creating a long format out of lists in genre column
movie_genre['genre'] = movie_genre['genre'].apply(lambda x: x.split(",") if type(x)==str else x)
movie_genre = movie_genre.explode('genre')

In [ ]:
# Reducing columns and cleaning up
movie_genre = movie_genre.drop(['title'], axis=1).reset_index(drop = True)
movie_genre = movie_genre.apply(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# List of genres in our database
genre_list = list(movie_genre['genre'].unique())
genre_list.sort()
genre_list

In [ ]:
# Creating dictionary with movie_IDs and mapping the dictionary on the genres column
genre_dict = {
    'Action': 1,
    'Adventure': 2,
    'Animation': 3,
    'Comedy': 4,
    'Crime': 5,
    'Documentary': 6,
    'Drama': 7,
    'Family': 8,
    'Fantasy': 9,
    'History': 10,
    'Horror': 11,
    'Music': 12,
    'Mystery': 13,
    'Other/unspecified': 14,
    'Romance': 15,
    'Science Fiction': 16,
    'TV Movie': 17,
    'Thriller': 18,
    'War': 19,
    'Western': 20
}
movie_genre['genre'] = movie_genre['genre'].apply(lambda x: genre_dict.get(x,x)) # With .get, if None is encountered, it returns key

In [ ]:
# Renaming column
movie_genre = movie_genre.rename(columns={'genre': 'genre_ID'})

In [ ]:
# Exporting movie_info table (link between movies and genres)
movie_genre.to_csv('movie_genre.csv', index=False)

In [ ]:
# Preparing genres table (all genres with unique ID)
genres = pd.DataFrame(
    {'genre_ID': [i+1 for i in range(len(genre_list))],
     'genre_name': genre_list
    })
genres

In [ ]:
# Exporting the genres table
genres.to_csv('genres.csv', index=False)